In [4]:
import math

In [8]:
def square_indices(n):
    max_idx = 2 * (n+1)
    res = [[0, x] for x in range(max_idx -1, -1, -1)]\
        + [[x, 0] for x in range(1, max_idx)]\
        + [[max_idx -1, x] for x in range(1, max_idx)]\
        + [[x, max_idx -1] for x in range(max_idx -2, 0, -1)]
    return res


square_indices(0)

[[0, 1], [0, 0], [1, 0], [1, 1]]

In [5]:
math.atan2(-5, 4)

-0.8960553845713439

In [6]:
math.degrees(0.8960553845713439)

51.34019174590991

In [15]:
for (x, y) in square_indices(0):
    angle = math.atan2(y -0.5, x -0.5)
    corr_angle =  angle
    if angle < 0:
        corr_angle = 2 * math.pi  - angle
    deg = math.degrees(angle)
    if deg < 0:
        deg += 360
    print([x, y], angle, corr_angle, deg)

[0, 1] 2.356194490192345 2.356194490192345 135.0
[0, 0] -2.356194490192345 8.63937979737193 225.0
[1, 0] -0.7853981633974483 7.0685834705770345 315.0
[1, 1] 0.7853981633974483 0.7853981633974483 45.0


In [17]:
# 0.0 0.5 1.0
# 0.5 
# 1.0  

# center 0.5, 0.5
# -0.5 -0.5
#  
[x * 90 - 45 for x in range(1, 5)]

[45, 135, 225, 315]

In [19]:
for (x, y) in square_indices(1):
    angle = math.atan2(y -0.5, x -0.5)
    corr_angle =  angle
    if angle < 0:
        corr_angle = 2 * math.pi  - angle
    deg = math.degrees(angle)
    if deg < 0:
        deg += 360
    print([x, y], deg, angle, corr_angle)

[0, 3] 101.30993247402021 1.7681918866447774 1.7681918866447774
[0, 2] 108.43494882292202 1.892546881191539 1.892546881191539
[0, 1] 135.0 2.356194490192345 2.356194490192345
[0, 0] 225.0 -2.356194490192345 8.63937979737193
[1, 0] 315.0 -0.7853981633974483 7.0685834705770345
[2, 0] 341.565051177078 -0.3217505543966422 6.604935861576228
[3, 0] 348.69006752597977 -0.19739555984988075 6.480580867029467
[3, 1] 11.309932474020213 0.19739555984988075 0.19739555984988075
[3, 2] 30.96375653207352 0.5404195002705842 0.5404195002705842
[3, 3] 45.0 0.7853981633974483 0.7853981633974483
[2, 3] 59.03624346792648 1.0303768265243125 1.0303768265243125
[1, 3] 78.69006752597979 1.373400766945016 1.373400766945016


In [20]:
dist(0, 3) 

2.5495097567963922

In [48]:
from math import (atan2, pi, sqrt)    

from collections import OrderedDict

import os
import numpy as np
import torch

torch.set_printoptions(sci_mode=False)

from src.polar_traversal import (
    Bridge,
    make_latitudes,
    get_start,
    gather_path,
    cumulative_distances,
)

import src.polar_traversal as polar_traversal


def get_dist(x, y):
    return sqrt((x-0.5) **2 + (y-0.5) ** 2)

def get_angle(x, y):
    angle = atan2(y -0.5, x -0.5)
    return angle if angle >= 0 else angle + 2 * pi


def get_sample(pos, normed, path_points, path_normals):    
    i = 1
    while i < len(normed):
        d0, d1 = normed[i-1], normed[i]
        if pos <= d1:
            ratio = (pos - d0) / (d1 - d0) 
            pnt = ratio * path_points[i-1] + (1 - ratio) * path_points[i]
            nrm = ratio * path_normals[i-1] + (1 - ratio) * path_normals[i]
            return pnt, nrm, i
        i += 1
    raise (pos, normed, path)

def single_path_sample(path, layer_no, total_layers):
    cumulative = cumulative_distances(path['points'])
    normed =  cumulative / cumulative[-1]
    pos = (layer_no+1) / (total_layers+1)
    pnt, nrm, _ = get_sample(pos, normed, 
                             path['points'], path['normals'])
    return pnt, nrm

def get_angles_series(n):
    return [x for x in range(0, n*2+1, 2)]

def get_shifted_latitudes(no):
    lats = make_latitudes(no)
    rot = len(lats) // 4
    return np.vstack([lats[rot:], lats[:rot]])

def get_sample(pos, normed, path_points, path_normals):    
    i = 1
    while i < len(normed):
        d0, d1 = normed[i-1], normed[i]
        if pos <= d1:
            ratio = (pos - d0) / (d1 - d0) 
            pnt = ratio * path_points[i-1] + (1 - ratio) * path_points[i]
            nrm = ratio * path_normals[i-1] + (1 - ratio) * path_normals[i]
            return pnt, nrm, i
        i += 1
    raise (pos, normed, path)

def single_path_sample(path, layer_no, total_layers):
    cumulative = cumulative_distances(path['points'])
    normed =  cumulative / cumulative[-1]
    pos = layer_no / total_layers    
    pnt, nrm, _ = get_sample(pos, normed, 
                             path['points'], path['normals'])
    return torch.tensor(pnt).float(), torch.tensor(nrm).float()

def square_indices(n):
    max_idx = 2 * (n+1)
    res = [[0, x] for x in range(max_idx -1, -1, -1)]\
        + [[x, 0] for x in range(1, max_idx)]\
        + [[max_idx -1, x] for x in range(1, max_idx)]\
        + [[x, max_idx -1] for x in range(max_idx -2, 0, -1)]
    return res

def shifted_square_indices(sq_no, square_no):    
    sq_ids =  square_indices(sq_no)
    ost = square_no - sq_no
    return [(i+ost, k+ost) for (i, k) in iter(sq_ids)]

def get_latitudes_angles(n):
    return (torch.arange(0, n) * (2 * pi / n)).tolist()

class Blueprint:
    
    def __init__(self, path, latitudes_num):
        self.path = path
        self.latitudes_num = latitudes_num
        self.bridge = Bridge(path)
        self.paths = self.gather_paths(latitudes_num)
        
    
    def gather_paths(self, latitudes_num=None):
        latitudes_num = latitudes_num or self.latitudes_num  
        latitude_angles = get_latitudes_angles(latitudes_num)        
        latitudes = make_latitudes(latitudes_num)
        bridge = self.bridge
        face_id, point, normal = get_start(bridge.mesh)
        paths = OrderedDict([])
        for i, (latitude, latitude_angle) in enumerate(zip(latitudes, latitude_angles)):
            print(i, latitudes_num)
            path = gather_path(latitude, face_id, point, normal, bridge)
            paths[latitude_angle] = path
        return paths
        
    def __repr__(self):
        return f'Blueprint: {self.path} latitudes_num: {self.latitudes_num}' 

    def grid_sample(self, square_no, save_path):
        side = (square_no + 1) * 2
        res = self.load_result(save_path) if save_path and os.path.exists(save_path) \
            else self.create_result(side)
            
#         points = torch.zeros(1, 3, side, side)
#         normals = torch.zeros(1, 3, side, side)
#         grid =  torch.zeros(side, side) - 9
        
        face_id, point, normal = get_start(self.bridge.mesh)
        for sq_no in range(square_no+1):
            print(sq_no, res['sq_no'][0])
            if sq_no > res['sq_no'][0]:                                
                side = (sq_no + 1) * 2
                sq_ids = shifted_square_indices(sq_no, square_no)            
                angles_no = len(sq_ids)
                latitudes = get_shifted_latitudes(angles_no)            
                for (r, c), latitude in zip(sq_ids, latitudes):
                    path = gather_path(latitude, face_id, point, normal, self.bridge)                
                    pnt, nrm = single_path_sample(path, sq_no, square_no)
                    res['points'][0, :, c, r] = pnt
                    res['normals'][0, :, c, r] = nrm                
                    res['grid'][c, r] = float(sq_no)                
                res['sq_no'][0] = sq_no
                self.save_result(res, save_path)
        return res            
        
    def create_result(self, side):
        return {
            'points': torch.zeros(1, 3, side, side),
            'normals': torch.zeros(1, 3, side, side),
            'grid': torch.zeros(side, side) - 9,
            'sq_no': torch.tensor([-1]),
        }
    
    def load_result(self, path):
        loaded = np.load(path)
        
        return {
            'points': torch.tensor(loaded['points']).float(),
            'normals': torch.tensor(loaded['normals']).float(),
            'grid': torch.tensor(loaded['grid']).float(),
            'sq_no': torch.tensor(loaded['sq_no']).long(),
        }
    
    def save_result(self, res, path):
        #assert type('s')  == 'str'
        print('path', path, res['sq_no'])
        np.savez_compressed(
            path,
            points=res['points'].numpy(),
            normals=res['normals'].numpy(),
            grid=res['grid'].numpy(),
            sq_no=res['sq_no'].numpy(),
        )
        

stl_path = './data/centered_3.stl'

square_no = 127
blueprint = Blueprint(stl_path, 4096)
print(blueprint)
torch.save(blueprint.paths, f'./data/paths_{len(blueprint.paths)}.pth')
# blueprint_path = './data/blueprint_radial{}.npz'.format(square_no)
# print(blueprint_path)
# res = blueprint.grid_sample(square_no, blueprint_path)
# res['points'].shape, res['normals'].shape

0 4096
1 4096
2 4096
3 4096
4 4096
5 4096
6 4096
7 4096
8 4096
9 4096
10 4096
11 4096
12 4096
13 4096
14 4096
15 4096
16 4096
17 4096
18 4096
19 4096
20 4096
21 4096
22 4096
23 4096
24 4096
25 4096
26 4096
27 4096
28 4096
29 4096
30 4096
31 4096
32 4096
33 4096
34 4096
35 4096
36 4096
37 4096
38 4096
39 4096
40 4096
41 4096
42 4096
43 4096
44 4096
45 4096
46 4096
47 4096
48 4096
49 4096
50 4096
51 4096
52 4096
53 4096
54 4096
55 4096
56 4096
57 4096
58 4096
59 4096
60 4096
61 4096
62 4096
63 4096
64 4096
65 4096
66 4096
67 4096
68 4096
69 4096
70 4096
71 4096
72 4096
73 4096
74 4096
75 4096
76 4096
77 4096
78 4096
79 4096
80 4096
81 4096
82 4096
83 4096
84 4096
85 4096
86 4096
87 4096
88 4096
89 4096
90 4096
91 4096
92 4096
93 4096
94 4096
95 4096
96 4096
97 4096
98 4096
99 4096
100 4096
101 4096
102 4096
103 4096
104 4096
105 4096
106 4096
107 4096
108 4096
109 4096
110 4096
111 4096
112 4096
113 4096
114 4096
115 4096
116 4096
117 4096
118 4096
119 4096
120 4096
121 4096
122 4096
123

923 4096
924 4096
925 4096
926 4096
927 4096
928 4096
929 4096
930 4096
931 4096
932 4096
933 4096
934 4096
935 4096
936 4096
937 4096
938 4096
939 4096
940 4096
941 4096
942 4096
943 4096
944 4096
945 4096
946 4096
947 4096
948 4096
949 4096
950 4096
951 4096
952 4096
953 4096
954 4096
955 4096
956 4096
957 4096
958 4096
959 4096
960 4096
961 4096
962 4096
963 4096
964 4096
965 4096
966 4096
967 4096
968 4096
969 4096
970 4096
971 4096
972 4096
973 4096
974 4096
975 4096
976 4096
977 4096
978 4096
979 4096
980 4096
981 4096
982 4096
983 4096
984 4096
985 4096
986 4096
987 4096
988 4096
989 4096
990 4096
991 4096
992 4096
993 4096
994 4096
995 4096
996 4096
997 4096
998 4096
999 4096
1000 4096
1001 4096
1002 4096
1003 4096
1004 4096
1005 4096
1006 4096
1007 4096
1008 4096
1009 4096
1010 4096
1011 4096
1012 4096
1013 4096
1014 4096
1015 4096
1016 4096
1017 4096
1018 4096
1019 4096
1020 4096
1021 4096
1022 4096
1023 4096
1024 4096
1025 4096
1026 4096
1027 4096
1028 4096
1029 4096
1030 40

1750 4096
1751 4096
1752 4096
1753 4096
1754 4096
1755 4096
1756 4096
1757 4096
1758 4096
1759 4096
1760 4096
1761 4096
1762 4096
1763 4096
1764 4096
1765 4096
1766 4096
1767 4096
1768 4096
1769 4096
1770 4096
1771 4096
1772 4096
1773 4096
1774 4096
1775 4096
1776 4096
1777 4096
1778 4096
1779 4096
1780 4096
1781 4096
1782 4096
1783 4096
1784 4096
1785 4096
1786 4096
1787 4096
1788 4096
1789 4096
1790 4096
1791 4096
1792 4096
1793 4096
1794 4096
1795 4096
1796 4096
1797 4096
1798 4096
1799 4096
1800 4096
1801 4096
1802 4096
1803 4096
1804 4096
1805 4096
1806 4096
1807 4096
1808 4096
1809 4096
1810 4096
1811 4096
1812 4096
1813 4096
1814 4096
1815 4096
1816 4096
1817 4096
1818 4096
1819 4096
1820 4096
1821 4096
1822 4096
1823 4096
1824 4096
1825 4096
1826 4096
1827 4096
1828 4096
1829 4096
1830 4096
1831 4096
1832 4096
1833 4096
1834 4096
1835 4096
1836 4096
1837 4096
1838 4096
1839 4096
1840 4096
1841 4096
1842 4096
1843 4096
1844 4096
1845 4096
1846 4096
1847 4096
1848 4096
1849 4096


2570 4096
2571 4096
2572 4096
2573 4096
2574 4096
2575 4096
2576 4096
2577 4096
2578 4096
2579 4096
2580 4096
2581 4096
2582 4096
2583 4096
2584 4096
2585 4096
2586 4096
2587 4096
2588 4096
2589 4096
2590 4096
2591 4096
2592 4096
2593 4096
2594 4096
2595 4096
2596 4096
2597 4096
2598 4096
2599 4096
2600 4096
2601 4096
2602 4096
2603 4096
2604 4096
2605 4096
2606 4096
2607 4096
2608 4096
2609 4096
2610 4096
2611 4096
2612 4096
2613 4096
2614 4096
2615 4096
2616 4096
2617 4096
2618 4096
2619 4096
2620 4096
2621 4096
2622 4096
2623 4096
2624 4096
2625 4096
2626 4096
2627 4096
2628 4096
2629 4096
2630 4096
2631 4096
2632 4096
2633 4096
2634 4096
2635 4096
2636 4096
2637 4096
2638 4096
2639 4096
2640 4096
2641 4096
2642 4096
2643 4096
2644 4096
2645 4096
2646 4096
2647 4096
2648 4096
2649 4096
2650 4096
2651 4096
2652 4096
2653 4096
2654 4096
2655 4096
2656 4096
2657 4096
2658 4096
2659 4096
2660 4096
2661 4096
2662 4096
2663 4096
2664 4096
2665 4096
2666 4096
2667 4096
2668 4096
2669 4096


3390 4096
3391 4096
3392 4096
3393 4096
3394 4096
3395 4096
3396 4096
3397 4096
3398 4096
3399 4096
3400 4096
3401 4096
3402 4096
3403 4096
3404 4096
3405 4096
3406 4096
3407 4096
3408 4096
3409 4096
3410 4096
3411 4096
3412 4096
3413 4096
3414 4096
3415 4096
3416 4096
3417 4096
3418 4096
3419 4096
3420 4096
3421 4096
3422 4096
3423 4096
3424 4096
3425 4096
3426 4096
3427 4096
3428 4096
3429 4096
3430 4096
3431 4096
3432 4096
3433 4096
3434 4096
3435 4096
3436 4096
3437 4096
3438 4096
3439 4096
3440 4096
3441 4096
3442 4096
3443 4096
3444 4096
3445 4096
3446 4096
3447 4096
3448 4096
3449 4096
3450 4096
3451 4096
3452 4096
3453 4096
3454 4096
3455 4096
3456 4096
3457 4096
3458 4096
3459 4096
3460 4096
3461 4096
3462 4096
3463 4096
3464 4096
3465 4096
3466 4096
3467 4096
3468 4096
3469 4096
3470 4096
3471 4096
3472 4096
3473 4096
3474 4096
3475 4096
3476 4096
3477 4096
3478 4096
3479 4096
3480 4096
3481 4096
3482 4096
3483 4096
3484 4096
3485 4096
3486 4096
3487 4096
3488 4096
3489 4096


In [47]:
2048*2

4096

In [41]:
len(blueprint.paths)

128

In [43]:
torch.save(blueprint.paths, f'./data/paths_{len(blueprint.paths)}.pth')

In [19]:
res = polar_traversal.get_start(blueprint.bridge.mesh)
res, len(res)

((5206, array([0.        , 0.        , 0.79351213]), array([0, 0, 1])), 3)

In [9]:
from collections import OrderedDict
d = OrderedDict([])
d

OrderedDict()

In [10]:
d[1] = [1]
d[2] = [2]
d

OrderedDict([(1, [1]), (2, [2])])

In [11]:
d.keys()

odict_keys([1, 2])

In [26]:
import torch
n = 4
r_angle = torch.stack((
        torch.zeros(n) + 2,
        torch.arange(0, n) * (2 * pi / n)))
r_angle

tensor([[2.0000, 2.0000, 2.0000, 2.0000],
        [0.0000, 1.5708, 3.1416, 4.7124]])

In [29]:
torch.arange(0, n) * (2 * pi / n)

tensor([0.0000, 1.5708, 3.1416, 4.7124])

In [33]:
r_angle.tolist()

[[2.0, 2.0, 2.0, 2.0],
 [0.0, 1.5707963705062866, 3.1415927410125732, 4.71238899230957]]